## Определение вида бабочки по изображению (Практическая часть)
**Задание базового уровня**

Вашим заданием в данной лабораторной будет построение модели на основе сверточных нейронных сетей для определения вида бабочки по ее фотографии.

Методика оценки - accuracy_score

В наборе данных представлены фотографии 100 видов бабочек. Все изображения имеют разрешение 224х224. Имя каждого файла состоит из названия вида бабочки и порядкового номера изображения.

**Задание повышенного уровня**

Выполнить задание базового уровня. Классификатор должен показывать точность по метрике accuracy_score на валидационных данных выше 80%.

In [1]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf
import PIL

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D, Dropout, Flatten

import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
import tensorflow.keras.layers as tfl

2024-04-28 10:38:54.558946: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-28 10:38:54.559057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-28 10:38:54.718003: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
train_dir='/kaggle/input/butterfly-images40-species/train'
test_dir='/kaggle/input/butterfly-images40-species/valid/'

In [3]:
image_size = 224
batch_size = 512
channel_size = 3 #RGB

num_classes = 100

In [4]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(    
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    
    horizontal_flip=True,
    fill_mode='nearest',
    zca_whitening=False,  
    vertical_flip=True,
    brightness_range=(0.5, 1.5),
    zoom_range=(0.5, 1.0) 
)

train_generator = datagen.flow_from_directory(
    train_dir,
    
    target_size=(image_size, image_size),
    batch_size=32,
    
    class_mode="categorical",
    
    shuffle=True,
    seed=42
)

Found 12594 images belonging to 100 classes.


In [5]:
validation_generator = datagen.flow_from_directory(
    test_dir,
    
    target_size=(image_size, image_size),
    batch_size=256,
    
    class_mode="categorical",
    
    shuffle=False,
    seed=42
)

Found 500 images belonging to 100 classes.


In [6]:
base_model = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3) , classes = 100 , pooling ='avg')

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(224, 224, 3)))
model.add(base_model)
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))


model.compile(
    optimizer=tf.keras.optimizers.Adam(0.01),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)


history = model.fit(
    train_generator,
    validation_data = validation_generator,
    epochs=10,
)

Epoch 1/10
  1/394 ━━━━━━━━━━━━━━━━━━━━ 2:47:24 26s/step - accuracy: 0.0312 - loss: 4.7248

W0000 00:00:1714310294.868371     102 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


315/394 ━━━━━━━━━━━━━━━━━━━━ 33s 430ms/step - accuracy: 0.5573 - loss: 1.8579

W0000 00:00:1714310429.859725     102 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


394/394 ━━━━━━━━━━━━━━━━━━━━ 0s 418ms/step - accuracy: 0.5821 - loss: 1.7281

W0000 00:00:1714310470.414002     101 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


394/394 ━━━━━━━━━━━━━━━━━━━━ 205s 456ms/step - accuracy: 0.5823 - loss: 1.7267 - val_accuracy: 0.5120 - val_loss: 2.1402


W0000 00:00:1714310474.055409     101 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Epoch 2/10
394/394 ━━━━━━━━━━━━━━━━━━━━ 176s 439ms/step - accuracy: 0.7601 - loss: 0.8286 - val_accuracy: 0.8360 - val_loss: 0.5816
Epoch 3/10
394/394 ━━━━━━━━━━━━━━━━━━━━ 212s 531ms/step - accuracy: 0.7836 - loss: 0.7549 - val_accuracy: 0.8240 - val_loss: 0.6755
Epoch 4/10
394/394 ━━━━━━━━━━━━━━━━━━━━ 176s 438ms/step - accuracy: 0.7938 - loss: 0.7285 - val_accuracy: 0.7540 - val_loss: 1.0819
Epoch 5/10
394/394 ━━━━━━━━━━━━━━━━━━━━ 166s 414ms/step - accuracy: 0.8006 - loss: 0.7069 - val_accuracy: 0.8420 - val_loss: 0.6215
Epoch 6/10
394/394 ━━━━━━━━━━━━━━━━━━━━ 168s 418ms/step - accuracy: 0.8059 - loss: 0.6886 - val_accuracy: 0.7840 - val_loss: 0.8446
Epoch 7/10
394/394 ━━━━━━━━━━━━━━━━━━━━ 167s 416ms/step - accuracy: 0.8200 - loss: 0.6295 - val_accuracy: 0.7420 - val_loss: 0.9423
Epoch 8/10
394/394 ━━━━━━━━━━━━━━━━━━━━ 166s 414ms/step - accuracy: 0.8224 - loss: 0.6369 - val_accuracy: 0.8340 - val_loss: 0.6331
Epoch 9/10
394/394 ━━━━━━━━━━━━━━━━━━━━ 169s 422ms/step - accuracy: 0.8371 -

In [11]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenet_1.00_224 (Functional) │ (None, 1024)           │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │       102,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,950,318 (37.96 MB)

 Trainable params: 3,309,476 (12.62 MB)

 Non-trainable params: 21,888 (85.50 KB)

 Optimizer params: 6,618,954 (25.25 MB)

In [12]:
base_model.summary()

Model: "mobilenet_1.00_224"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1 (Conv2D)                  │ (None, 112, 112, 32)   │           864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_bn (BatchNormalization)   │ (None, 112, 112, 32)   │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_relu (ReLU)               │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1 (DepthwiseConv2D)     │ (None, 112, 112, 32)   │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_bn                    │ (None, 112, 112, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_1_relu (ReLU)           │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1 (Conv2D)              │ (None, 112, 112, 64)   │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_bn                    │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_1_relu (ReLU)           │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pad_2 (ZeroPadding2D)      │ (None, 113, 113, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2 (DepthwiseConv2D)     │ (None, 56, 56, 64)     │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_bn                    │ (None, 56, 56, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_2_relu (ReLU)           │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2 (Conv2D)              │ (None, 56, 56, 128)    │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_2_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3 (DepthwiseConv2D)     │ (None, 56, 56, 128)    │         1,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_dw_3_relu (ReLU)           │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3 (Conv2D)              │ (None, 56, 56, 128)    │        16,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_pw_3_bn                    │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │             

 Total params: 3,228,864 (12.32 MB)

 Trainable params: 3,206,976 (12.23 MB)

 Non-trainable params: 21,888 (85.50 KB)

Предобученная нейросеть без предобработки изображений показывает результат **~80 процентов**          
Если добавить augmentation с зумом, повортом, изменением контраста, то можно получить ~90 процентов